In [1]:
import sys
import os
import numpy as np
import pandas as pd

# Menambahkan path ke folder `src`
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.features.featuresExtraction import FeaturesExtraction
from src.features.audioFilter import filter_audio

c:\Users\mmuuh\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def ExtractFromFolder(list, folderName, var):
    for folderNames in os.listdir(folderName):
        for fileNames in os.listdir(folderName + folderNames):
            if fileNames.endswith(".wav"):
                features = FeaturesExtraction(path=folderName + folderNames, audio_file=fileNames, target=folderNames)
                if var == 1:
                    list[0][folderNames].append(features)
                elif var == 2:
                    list[1][folderNames].append(features)
                elif var == 3:
                    list[2][folderNames].append(features)
    
    return list


In [3]:
__root_folder__ = ["../data1/template/p1/","../data1/template/p2/","../data1/template/p3/"]

d1_p1_tem = {'a': [], 'e': [], 'i': [], 'o': [], 'u': []}
d1_p2_tem = {'a': [], 'e': [], 'i': [], 'o': [], 'u': []}
d1_p3_tem = {'a': [], 'e': [], 'i': [], 'o': [], 'u': []}

d1_list_tem = [d1_p1_tem, d1_p2_tem, d1_p3_tem]

for index, root in enumerate(__root_folder__):
    [d1_p1_tem, d1_p2_tem, d1_p3_tem] = ExtractFromFolder(d1_list_tem, root, index+1)

In [4]:
# p1_filtered = {
#     'a': [],
#     'e': [],
#     'i': [],
#     'o': [],
#     'u': []
# }

# p2_filtered = {
#     'a': [],
#     'e': [],
#     'i': [],
#     'o': [],
#     'u': []
# }

# p3_filtered = {
#     'a': [],
#     'e': [],
#     'i': [],
#     'o': [],
#     'u': []
# }

# __root_header__ = "../data/"

# def FilterAudio(root, index):
#     for folderNames in os.listdir(__root_header__ + root):
#         for fileNames in os.listdir(__root_header__ + root + folderNames):
#             if fileNames.endswith(".wav"):
#                 # print("Processing: ", fileNames)
#                 filter_audio(input=__root_header__ + root + folderNames+"/"+fileNames, output="../tmpFiltered/"+ root + fileNames)
#                 features = FeaturesExtraction(path="../tmpFiltered/" + root, audio_file=fileNames, target=folderNames)
#                 if index == 1:
#                     p1_filtered[folderNames].append(features)
#                 elif index == 2:
#                     p2_filtered[folderNames].append(features)
#                 elif index == 3:
#                     p3_filtered[folderNames].append(features)

# __root_folder__ = ["p1/","p2/","p3/"]

# for index, root in enumerate(__root_folder__):
#     FilterAudio(root, index+1)

In [5]:
vowels = ['a','e','i','o','u']
mean_feat= {
    'a': [],
    'e': [],
    'i': [],
    'o': [],
    'u': []
}

dataset = [d1_p1_tem, d1_p2_tem, d1_p3_tem]
for vowel in vowels:
    feat_store = []
    for data in dataset:
        for feature in data[vowel]:
            mfcc_mean = feature.mfccs.get_mean_features()
            feat_store.append(mfcc_mean)

    feat_store = np.array(feat_store)
    mean = np.mean(feat_store, axis=0)
    mean_feat[vowel] = mean

In [6]:
from src.features.dtw import DTW, VowelIdentifier

vi_mean = VowelIdentifier()

for vowel in vowels:
    vi_mean.add_reference(vowel, mean_feat[vowel])
    

# TESTING

In [7]:
# sample = FeaturesExtraction(path = '../data/p1/i/', audio_file='I0012.wav', target='i/')
# sample_mean = sample.mfccs.get_mean_features()
# identified_vowel, distance = vi_mean.identify(sample_mean)

# print(f"Identified Vowel: {identified_vowel}, DTW Distance: {distance}")
# dtw_instance = vi_mean.dtw
# distance, path = dtw_instance.dtw(sample_mean, vi_mean.reference_features[identified_vowel])
# dtw_instance.plot_cost_matrix(title=f"DTW Cost Matrix for {identified_vowel}")


In [8]:
def identifyVowel(feature: FeaturesExtraction, v: VowelIdentifier):
    feature_mean = feature.mfccs.get_mean_features()
    identified_vowel, distance = v.identify(feature_mean)
    return identified_vowel, distance

## Testing & Accuracy

In [9]:
__root_folder__ = ["../data1/testing/p1/","../data1/testing/p2/","../data1/testing/p3/"]

d1_p1_tes = {'a': [], 'e': [], 'i': [], 'o': [], 'u': []}
d1_p2_tes = {'a': [], 'e': [], 'i': [], 'o': [], 'u': []}
d1_p3_tes = {'a': [], 'e': [], 'i': [], 'o': [], 'u': []}

d1_list_tes = [d1_p1_tes, d1_p2_tes, d1_p3_tes]

for index, root in enumerate(__root_folder__):
    [d1_p1_tes, d1_p2_tes, d1_p3_tes] = ExtractFromFolder(d1_list_tes, root, index+1)


In [10]:
## P1 : Hanan

correct_count = 0
total = 0

for vowel in vowels:
    total_vowel = 0
    correct_vowel = 0
    for feature in d1_p1_tes[vowel]:
        identified_vowel, distance = identifyVowel(feature=feature, v=vi_mean)
        # print(f"Vowel: {vowel}, Identified Vowel: {identified_vowel}, DTW Distance: {distance}")
        total += 1
        total_vowel += 1
        if (identified_vowel[0] == vowel):
            correct_count += 1
            correct_vowel += 1
            # print("Correct Identification")
        # else:
            # print("False Identification")

    print(f"Correct Identification for {vowel}: {correct_vowel}")
    print(f"From total: {total_vowel}")
    print(f"Accuracy for {vowel}: {correct_vowel/total_vowel}")
    print()

print("P1 : Hanan -> Accurate 1")
print(f"Correct Identification: {correct_count}")
print(f"From total: {total}")
print(f"Accuracy: {correct_count/total}")

Correct Identification for a: 7
From total: 9
Accuracy for a: 0.7777777777777778

Correct Identification for e: 9
From total: 17
Accuracy for e: 0.5294117647058824

Correct Identification for i: 13
From total: 13
Accuracy for i: 1.0

Correct Identification for o: 2
From total: 16
Accuracy for o: 0.125

Correct Identification for u: 14
From total: 15
Accuracy for u: 0.9333333333333333

P1 : Hanan -> Accurate 1
Correct Identification: 45
From total: 70
Accuracy: 0.6428571428571429


In [11]:
## P2 : Alex

correct_count = 0
total = 0

for vowel in vowels:
    total_vowel = 0
    correct_vowel = 0
    for feature in d1_p2_tes[vowel]:
        identified_vowel, distance = identifyVowel(feature=feature, v=vi_mean)
        # print(f"Vowel: {vowel}, Identified Vowel: {identified_vowel}, DTW Distance: {distance}")
        total += 1
        total_vowel += 1
        if (identified_vowel[0] == vowel):
            correct_count += 1
            correct_vowel += 1
        #     print("Correct Identification")
        # else:
        #     print("False Identification")

    print(f"Correct Identification for {vowel}: {correct_vowel}")
    print(f"From total: {total_vowel}")
    print(f"Accuracy for {vowel}: {correct_vowel/total_vowel}")
    print()


print("P2 : Alex -> Accurate 1")
print(f"Correct Identification: {correct_count}")
print(f"From total: {total}")
print(f"Accuracy: {correct_count/total}")

Correct Identification for a: 9
From total: 10
Accuracy for a: 0.9

Correct Identification for e: 11
From total: 11
Accuracy for e: 1.0

Correct Identification for i: 2
From total: 10
Accuracy for i: 0.2

Correct Identification for o: 11
From total: 12
Accuracy for o: 0.9166666666666666

Correct Identification for u: 11
From total: 11
Accuracy for u: 1.0

P2 : Alex -> Accurate 1
Correct Identification: 44
From total: 54
Accuracy: 0.8148148148148148


In [12]:
## P3 : Rifqi

correct_count = 0
total = 0

for vowel in vowels:
    total_vowel = 0
    correct_vowel = 0
    for feature in d1_p3_tes[vowel]:
        identified_vowel, distance = identifyVowel(feature=feature, v=vi_mean)
        # print(f"Vowel: {vowel}, Identified Vowel: {identified_vowel}, DTW Distance: {distance}")
        total += 1
        total_vowel += 1
        if (identified_vowel[0] == vowel):
            correct_count += 1
            correct_vowel += 1
        #     print("Correct Identification")
        # else:
        #     print("False Identification")

    print(f"Correct Identification for {vowel}: {correct_vowel}")
    print(f"From total: {total_vowel}")
    print(f"Accuracy for {vowel}: {correct_vowel/total_vowel}")
    print()


print("P3 : Rifqi -> Accurate 1")
print(f"Correct Identification: {correct_count}")
print(f"From total: {total}")
print(f"Accuracy: {correct_count/total}")

Correct Identification for a: 3
From total: 13
Accuracy for a: 0.23076923076923078

Correct Identification for e: 12
From total: 15
Accuracy for e: 0.8

Correct Identification for i: 13
From total: 14
Accuracy for i: 0.9285714285714286

Correct Identification for o: 5
From total: 13
Accuracy for o: 0.38461538461538464

Correct Identification for u: 8
From total: 14
Accuracy for u: 0.5714285714285714

P3 : Rifqi -> Accurate 1
Correct Identification: 41
From total: 69
Accuracy: 0.5942028985507246
